# Snowflake on Snowflake: Automated Marketing Lead Screening Pipelines

This is an example of Snowflake's own marketing analytics team leveerages Snowflake's Cortex AISQL to analyze, cleanse, and enrich incoming marketing leads captured from a web form. 

The process filters out spam and harmful content, scores leads based on their fit with the Ideal Customer Profile (ICP), and categorizes them by job seniority. 

The primary business value is the creation of a more efficient and intelligent lead management workflow; this automation saves significant manual effort, enables sales and marketing teams to prioritize the highest-potential prospects, and allows for personalized outreach at scale, ultimately accelerating the sales cycle and increasing conversion rates.


In [ ]:
select * from ai_sql_demo_db.MARKETING_LEADS.marketing_form_data;

First step, we want to filter out irrelevant or harmful information user filled.

In [ ]:
----------------------------------------------------------------------------------------------------
--- filter out Spam Records
----------------------------------------------------------------------------------------------------

CREATE OR REPLACE TEMP TABLE MARKETING_LEADS_FILTERED AS
SELECT d.*
FROM ai_sql_demo_db.MARKETING_LEADS.marketing_form_data d
WHERE AI_FILTER('We are a B2B SAAS company, and this is the text entered to a demand gen form on our website to register for a marketing event. Please confirm if this is a form fill that appears to be legitimate data from a valid persona with a company, without any junk, spam, scams, jokes, or nonsensical entries that cannot be worked by sales. Please do not include students:' || concat_ws(' '
            ,'First Name:', d.first_name
            ,'Last Name:', d.last_name
            ,'Job Title:', d.title
            ,'Company:', d.company ))
  ;

Next step, we will classify each entry into "High", "Medium", "Low", "Poor" in terms of leads quality to identify ICPs (Ideal Customer Profile)

In [ ]:
----------------------------------------------------------------------------------------------------
--- Add Ideal Customer Profile
----------------------------------------------------------------------------------------------------

CREATE OR REPLACE TEMP TABLE MARKETING_LEADS_W_QUALITY AS

select d.*
     , AI_CLASSIFY(
        concat_ws(' '
        ,'Job Title:', d.title
        ,'Company:', d.company ),
   [{
    'label': 'High',
    'description': 'The lead has a decision-making role, relevant persona, and comes from a company well-aligned with B2B SAAS target industries and size. They must have a clear business need for data and AI services.'
    },{
    'label': 'Medium',
    'description': 'The lead is a good fit but may lack full decision-making authority or strong company characteristics. However, they should have influence or future potential.'
    },{
    'label': 'Low',
    'description': 'The lead has minimal alignment, lacks decision-making power, or is from a less relevant company. The persona or company factors do not strongly align with B2B SAAS ICP.'
    },{
    'label': 'Poor',
    'description': 'The lead has no alignment with B2B SAAS ICP. Their role and company characteristics are irrelevant. A form fill that appears to contain test accounts, junk, spam, scams, jokes, or nonsensical entries should be labeled as poor.'
    }
    ],
  {'task_description': 'We are a B2B SAAS company. Return a classification the Ideal Customer Profile of this lead based on the text entered to a demand gen form on our website to register for a marketing event.'}
  ) as classification_output_raw
  , classification_output_raw:labels[0]::text as leads_quality
from MARKETING_LEADS_FILTERED d;


In [ ]:
select * from MARKETING_LEADS_W_QUALITY;

Related, we can also leverage AI_CLASSIFY to add a column upon the lead's persona

In [ ]:
----------------------------------------------------------------------------------------------------
--- Add seniority persona field 
----------------------------------------------------------------------------------------------------

select d.*
     , AI_CLASSIFY(
            concat_ws(' '
            ,'Job Title:', d.title
            ),
       [{
        'label': 'CXO',
        'description': 'Any title that is a company C-level executive or founder.'
        },{
        'label': 'VP+',
        'description': 'Any title that is a company executive below C-level like a VP, President, or managing director.'
        },{
        'label': 'Director',
        'description': 'Any title that is director level or head of a department.'
        },{
        'label': 'Manager',
        'description': 'Any title that relates to managers or team leads.'
        },{
        'label': 'IC',
        'description': 'Any title that relates to an individual contributor.'
        }
        ,{
        'label': 'Junk',
        'description': 'Any title seems like junk, spam, scams, jokes, or nonsensical entries that cannot be worked by sales'
        }
        ],
      {'task_description': 'We are a B2B SAAS company. Use this data to classify job titles into seniority groupings. Consider all parts of the title and be careful of how the meaning changes based on parathesis or other punctuation.'}
      ) as classification_output_raw
      , classification_output_raw:labels[0]::text as classification_output
from MARKETING_LEADS_FILTERED d;

;

Lastly, for the high quality leads, I can also join in other company specific information, to help me draft a specific email related to customer's use case to increase the response rate.

In [ ]:
with cte as (
    select * from MARKETING_LEADS_W_QUALITY
    left join company_info using (company)
    where leads_quality = 'High'
)
select 
    *, 
    ai_complete(
        'llama4-maverick',
        prompt('We are a B2B SAAS company focusing on data and analytics platform. And this is a form filled in by our potential prospect. Can you help me draft an outreach email to ask if any interest in how data and analytics might supercharge their business. Please only return the email draft and nothing else. Here is the filled form: {0}; \nhere is the company info: {1}',
    concat_ws(' '
            ,'First Name:', first_name
            ,'Last Name:', last_name
            ,'Job Title:', title
            ,'Company:', company ),
            coalesce(description,''))
            )
from cte;

Similarly, we leverage a similar AI-powered prospecting system to revolutionalize proactive outreach, that we leverage AI and combining information from various sources, filter out high quality lead and curate customized outreach message to maximize answer rate.


Link to architecture flow chart: [LINK](https://docs.google.com/presentation/d/1WIg--DfZYs2B4_1p4JYcdqBCeppLa91GYC5rK92xS0s/edit?slide=id.g3692edc8cc6_0_175#slide=id.g3692edc8cc6_0_175)